In [1]:
import os
from functools import reduce
import pickle
import pandas as pd

In [2]:
#################################
WORK_DIR = 'C:/Users/school/Desktop/DATA_PREP_DIR/'
RECALC_DATA = False # refreshes data from scratch when True, reads pickles when False
#################################

# Directories that exist
SF_DIR = os.path.join(WORK_DIR, '2019_acs/sf')
SF_TEMPLATE_DIR = os.path.join(WORK_DIR, '2019_acs/sf_template')

# Directories created by this script
INT_DIR = os.path.join(WORK_DIR, 'INTERMEDIATE_DATA')
RES_DIR = os.path.join(WORK_DIR, 'RESULTS/')

try:
    os.makedirs(INT_DIR)
except Exception as e:
    print(e)
    
try:
    os.makedirs(RES_DIR)
except Exception as e:
    print(e)

[WinError 183] Cannot create a file when that file already exists: 'C:/Users/school/Desktop/DATA_PREP_DIR/INTERMEDIATE_DATA'
[WinError 183] Cannot create a file when that file already exists: 'C:/Users/school/Desktop/DATA_PREP_DIR/RESULTS/'


In [3]:
def save_pkl(file_path, data):
    with open(file_path, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def read_pkl(file_path):
    with open(file_path, 'rb') as handle:
        data = pickle.load(handle)
        return data

Summary files start with e, m, or g, indicating:

* e: estimates
* m: margins of error
* g: geography files

In [4]:
sf_files = os.listdir(SF_DIR)
# verify present files fall into the three categories, as denoted by first letter of filename
for first_letter in [x[0] for x in sf_files]:
    assert first_letter in ['e', 'm', 'g']

#### Build 'Estimates' Data Dict

In [5]:
# Build data dictionary of encoded column name: description
SF_DATA_DICT_dict = {}
for f in os.listdir(SF_TEMPLATE_DIR):
    if not f.startswith('seq'):
        continue
    file_path = os.path.join(SF_TEMPLATE_DIR, f)
    template_data = pd.read_excel(file_path, sheet_name='e')
    mapping = dict(zip(template_data.columns, template_data.iloc[0]))
    SF_DATA_DICT_dict.update(mapping)
SF_DATA_DICT = pd.Series(SF_DATA_DICT_dict).reset_index()
SF_DATA_DICT.columns = ['column', 'description']
SF_DATA_DICT.to_pickle(os.path.join(INT_DIR, 'SF_DATA_DICT.pkl'))
SF_DATA_DICT

,column,description
0,FILEID,FILEID
1,FILETYPE,FILETYPE
2,STUSAB,STUSAB
3,CHARITER,CHARITER
4,SEQUENCE,SEQUENCE
...,...,...
35528,C23002C_023,SEX BY AGE BY EMPLOYMENT STATUS FOR THE POPULA...
35529,C23002C_024,SEX BY AGE BY EMPLOYMENT STATUS FOR THE POPULA...
35530,C23002C_025,SEX BY AGE BY EMPLOYMENT STATUS FOR THE POPULA...
35531,C23002C_026,SEX BY AGE BY EMPLOYMENT STATUS FOR THE POPULA...


#### Build 'Estimates' table

In [6]:
# First, prepare dict of {<int>seq: column names} for estimates to speed up lookups
COL_NAME_LU = {}
for template in os.listdir(SF_TEMPLATE_DIR):
    if not template.startswith('seq'):
        continue
    seq = int(template.split('.')[0][3:])
    columns = pd.read_excel(os.path.join(SF_TEMPLATE_DIR, template), sheet_name='e').columns
    COL_NAME_LU[seq] = columns

In [7]:
if RECALC_DATA:
    # build dataframes by state, then later all data will be concatenated
    # This makes things more efficient due to the large volume of tables.
    RESULTS = {}
    sf_e_files = [x for x in sf_files if x[0]=='e']
    for i, f in enumerate(sf_e_files):
        print('\rPercent complete: {}%'.format(round((float(i+1)/len(sf_e_files))*100., 2)), end='')
        # get filetype
        ftype = f[0]
        # get path to file for reading
        file_path = os.path.join(SF_DIR, f)
        # get state
        state = f[6:8]
        # get sequence
        seq = int(f[8:12])
        # look up column names for this file
        columns = COL_NAME_LU[seq]
        # read the data and set the appropriate column names
        data = pd.read_csv(file_path, header=None, names=columns)
        # drop unnecessary columns
        data.drop(columns=['FILEID', 'FILETYPE', 'CHARITER', 'SEQUENCE'], inplace=True)
        # set index for concatenation - needed because `.merge()` on
        # this many files takes FOREVER.
        data.set_index(['STUSAB','LOGRECNO'], inplace=True)

        # merge data for current state
        if state not in RESULTS.keys():
            RESULTS[state] = data
        else:
            RESULTS[state] = pd.concat([RESULTS[state], data], axis=1)


    save_pkl(os.path.join(INT_DIR, 'RESULTS.pkl'), RESULTS)
    print('\nDone!\nNumber of states processed: {}'.format(len(RESULTS)))
else:
    print('Loading previous result...')
    RESULTS = read_pkl(os.path.join(INT_DIR, 'RESULTS.pkl'))
    print('Done')

Loading previous result...
Done


In [8]:
print('Concatenating results...')

if RECALC_DATA:
    # Reset indicies
    for i, (state, df) in enumerate(RESULTS.items()):
        df.reset_index(inplace=True)

    # Concatenate tables from different states
    ESTIMATES = pd.concat([df for df in RESULTS.values()])
    ESTIMATES.to_pickle(os.path.join(INT_DIR, 'ESTIMATES.pkl'))
else:
    print('Loading previous result...')
    ESTIMATES = pd.read_pickle(os.path.join(INT_DIR, 'ESTIMATES.pkl'))
    print('Done')    

del RESULTS # free up some memory, we don't need those anymore.
ESTIMATES

Concatenating results...
Loading previous result...
Done


,STUSAB,LOGRECNO,B01001_001,B01001_002,B01001_003,B01001_004,B01001_005,B01001_006,B01001_007,B01001_008,...,B99282_005,B99282_006,B99282_007,B99282_008,B99282_009,B99283_001,B99283_002,B99283_003,B99283_004,B99283_005
0,ak,1,731545.0,380433.0,24910.0,28292.0,25256.0,14719.0,11147.0,5181.0,...,248183,3838,248361,3849,248350,252199,7194,7194,10098,239283
1,ak,2,474992.0,240654.0,15669.0,17866.0,15164.0,9242.0,7846.0,3480.0,...,166396,2680,166398,2693,166385,169078,4779,4779,7555,159895
2,ak,3,256553.0,139779.0,9241.0,10426.0,10092.0,5477.0,3301.0,1701.0,...,81787,1158,81963,1156,81965,83121,2415,2415,2543,79388
3,ak,4,539488.0,277116.0,17803.0,21038.0,17743.0,10793.0,8374.0,4095.0,...,188899,3050,188904,3079,188875,191954,5649,5649,8298,181453
4,ak,5,493166.0,254154.0,16238.0,19841.0,16232.0,9966.0,8068.0,3884.0,...,170727,2790,170700,2806,170684,173490,5312,5312,7780,163671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,wy,17,138380.0,71563.0,3983.0,5431.0,3128.0,2544.0,2638.0,1927.0,...,56945,563,56945,623,56885,57508,1296,1296,3628,53564
17,wy,18,108730.0,54849.0,3410.0,3336.0,4853.0,2200.0,1222.0,869.0,...,45296,339,45296,449,45186,45635,1145,1145,2471,42299
18,wy,19,118373.0,60147.0,4488.0,5277.0,4274.0,2402.0,1923.0,971.0,...,42101,62,42101,230,41933,42163,652,652,1926,39900
19,wy,20,91520.0,46495.0,2777.0,3807.0,2237.0,1636.0,1164.0,599.0,...,38639,332,38639,392,38579,38971,1036,1036,2968,35756


## Geo Data

In [9]:
# The MINI GEO file appears to be a subset of the data contained in the 'g' files.
# Therefore, this file will be ignored for now and the 'g' files used instead.

# GEO_PATH = os.path.join(WORK_DIR, '2019_acs/geo/1_year_Mini_Geo.xlsx')
# geo = pd.read_excel(GEO_PATH, sheet_name=None)
# geo[list(geo.keys())[5]]

# # Combine all sheets from geo file into single table
# for i, (sheet, data) in enumerate(geo.items()):
#     print('\rPercent complete: {}%'.format(round((float(i+1)/len(geo))*100., 2)), end='')
#     # rename columns so state and log rec no are the same names
#     data.rename(columns={
#         'State': 'STUSAB',
#         'Logical Record Number': 'LOGRECNO',
#         'Geography ID': 'GEOID',
#         'Geography Name': 'GEONAME'}, inplace=True)
#     # set the state abbrev to lowercase to match the summary data for the join
#     data.STUSAB = data.STUSAB.str.strip().str.lower()

# geo_prep = pd.concat(geo.values())
# geo_prep

#### Build 'Geo' Data Dict

In [10]:
# Build data dictionary of encoded column name: description
file_path = os.path.join(SF_TEMPLATE_DIR, '2019_SFGeoFileTemplate.xlsx')
template_data = pd.read_excel(file_path)
GEO_COLS = template_data.columns
GEO_DATA_DICT_dict = dict(zip(GEO_COLS, template_data.iloc[0]))
    
GEO_DATA_DICT = pd.Series(GEO_DATA_DICT_dict).reset_index()
GEO_DATA_DICT.columns = ['column', 'description']
GEO_DATA_DICT.to_pickle(os.path.join(INT_DIR, 'SF_DATA_DICT.pkl'))
GEO_DATA_DICT.head(10)

,column,description
0,FILEID,Always equal to ACS Summary File identification
1,STUSAB,State Postal Abbreviation
2,SUMLEVEL,Summary Level
3,COMPONENT,Geographic Component
4,LOGRECNO,Logical Record Number
5,US,US
6,REGION,Census Region
7,DIVISION,Census Division
8,STATECE,State (Census Code)
9,STATE,State (FIPS Code)


In [11]:
# if RECALC_DATA:
#     # Merge geo data into ESTIMATES
#     ESTIMATES_GEO = ESTIMATES.sort_index(axis=1).merge(geo_prep, on=['STUSAB', 'LOGRECNO'], how='left')

#     # reorder columns
#     info_cols = ['STUSAB', 'LOGRECNO', 'GEOID', 'GEONAME']
#     all_other_cols = sorted(list(set(ESTIMATES_GEO.columns)-set(info_cols)))
#     ESTIMATES_GEO = ESTIMATES_GEO[info_cols + all_other_cols]
#     ESTIMATES_GEO.to_pickle(os.path.join(INT_DIR, 'ESTIMATES_GEO.pkl'))
# else:
#     print('Loading previous result...')
#     ESTIMATES_GEO = pd.read_pickle(os.path.join(INT_DIR, 'ESTIMATES_GEO.pkl'))
#     print('Done')        

# ESTIMATES_GEO.head()

In [12]:
if RECALC_DATA:
    GEO_TABLE = pd.DataFrame()
    sf_g_files = [x for x in sf_files if x[0]=='g' and x.endswith('.csv')]
    columns = GEO_COLS

    for i, f in enumerate(sf_g_files):
        print('\rPercent complete: {}%'.format(round((float(i+1)/len(sf_g_files))*100., 2)), end='')
        # get filetype
        ftype = f[0]
        # get path to file for reading
        file_path = os.path.join(SF_DIR, f)
        # get state
        state = f.split('.')[0][-2:]

        # read the data and set the appropriate column names
        try:
            data = pd.read_csv(file_path, header=None, names=columns, encoding="ISO-8859-1")
        except Exception as e:
            print(e)
            print('\nFailure on file {}\n'.format(f))
            continue

        GEO_TABLE = GEO_TABLE.append(data)


    save_pkl(os.path.join(INT_DIR, 'GEO_TABLE.pkl'), GEO_TABLE)
    print('\nDone!')
else:
    print('Loading previous result...')
    GEO_TABLE = read_pkl(os.path.join(INT_DIR, 'GEO_TABLE.pkl'))
    print('Done')
    
GEO_TABLE = GEO_TABLE[['STUSAB', 'LOGRECNO']+list(set(GEO_TABLE.columns)-set(['STUSAB', 'LOGRECNO']))]
GEO_TABLE.STUSAB = GEO_TABLE.STUSAB.str.strip().str.lower()
GEO_TABLE.columns = [x.replace('.', '') for x in GEO_TABLE.columns]
GEO_TABLE

Loading previous result...
Done


,STUSAB,LOGRECNO,NECTADIV,BLANK1,CDCURR,PCI,COMPONENT,COUSUB,US,PLACE,...,AIHHTLI,STATE,COUNTY,BLKGRP,ZCTA5,STATECE,AITSCE,REGION,MEMI,BLANK4
0,ak,1,NaN,NaN,NaN,NaN,00,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ak,2,NaN,NaN,NaN,NaN,01,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ak,3,NaN,NaN,NaN,NaN,43,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ak,4,NaN,NaN,NaN,NaN,A0,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ak,5,NaN,NaN,NaN,NaN,C0,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,wy,17,NaN,NaN,NaN,NaN,00,NaN,NaN,NaN,...,NaN,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,wy,18,NaN,NaN,NaN,NaN,00,NaN,NaN,NaN,...,NaN,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,wy,19,NaN,NaN,NaN,NaN,00,NaN,NaN,NaN,...,NaN,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,wy,20,NaN,NaN,NaN,NaN,00,NaN,NaN,NaN,...,NaN,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
GEO_TABLE.iloc[2]

STUSAB                    ak
LOGRECNO                   3
NECTADIV                 NaN
BLANK1                   NaN
CDCURR                   NaN
PCI                      NaN
COMPONENT                 43
COUSUB                   NaN
US                       NaN
PLACE                    NaN
SDUNI                    NaN
MACC                     NaN
BLANK2                   NaN
METDIV                   NaN
AIANHH                   NaN
CSA                      NaN
BLANK3                   NaN
NAME         Alaska -- Rural
SLDL                     NaN
GEOID              04043US02
BLANK6                   NaN
CNECTA                   NaN
FILEID                 ACSSF
CONCIT                   NaN
BLANK                    NaN
SLDU                     NaN
AIANHHFP                 NaN
UA                       NaN
UR                         R
PUMA5                    NaN
BLANK5                   NaN
SUMLEVEL                  40
SDSEC                    NaN
DIVISION                 NaN
AITS          

---

#### Gazeteer File

In [14]:
GAZ_TABLE = pd.read_csv(os.path.join(WORK_DIR, '2019_acs/gaz_tract/2019_Gaz_tracts_national.txt'), sep='\t')
GAZ_TABLE.rename(columns={'USPS': 'STUSAB'}, inplace=True)
GAZ_TABLE.STUSAB = GAZ_TABLE.STUSAB.str.strip().str.lower()
GAZ_TABLE

,STUSAB,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,al,1001020100,9817813,28435,3.791,0.011,32.481959,-86.491338
1,al,1001020200,3325680,5669,1.284,0.002,32.475758,-86.472468
2,al,1001020300,5349273,9054,2.065,0.003,32.474024,-86.459703
3,al,1001020400,6384276,8408,2.465,0.003,32.471030,-86.444835
4,al,1001020500,11408866,43534,4.405,0.017,32.458922,-86.421826
...,...,...,...,...,...,...,...,...
73996,pr,72153750501,1820185,0,0.703,0.000,18.031211,-66.867347
73997,pr,72153750502,689930,0,0.266,0.000,18.024746,-66.860442
73998,pr,72153750503,3298433,1952,1.274,0.001,18.023148,-66.876603
73999,pr,72153750601,10987037,4527,4.242,0.002,18.017808,-66.839070


In [15]:
set(GAZ_TABLE.GEOID.values).intersection(set(GEO_TABLE.GEOID.values))

set()

In [16]:
for state in GAZ_TABLE.STUSAB.unique():
    print('')
    print(state)
    print('GAZ: {}'.format(GAZ_TABLE[GAZ_TABLE.STUSAB==state].GEOID.nunique()))
    print('GEO: {}'.format(GEO_TABLE[GEO_TABLE.STUSAB==state].GEOID.nunique()))


al
GAZ: 1181
GEO: 105

ak
GAZ: 167
GEO: 26

az
GAZ: 1526
GEO: 149

ar
GAZ: 686
GEO: 71

ca
GAZ: 8057
GEO: 796

co
GAZ: 1249
GEO: 120

ct
GAZ: 833
GEO: 86

de
GAZ: 218
GEO: 24

dc
GAZ: 179
GEO: 15

fl
GAZ: 4245
GEO: 362

ga
GAZ: 1969
GEO: 193

hi
GAZ: 351
GEO: 29

id
GAZ: 298
GEO: 43

il
GAZ: 3123
GEO: 268

in
GAZ: 1511
GEO: 162

ia
GAZ: 825
GEO: 75

ks
GAZ: 770
GEO: 73

ky
GAZ: 1115
GEO: 81

la
GAZ: 1148
GEO: 101

me
GAZ: 358
GEO: 34

md
GAZ: 1406
GEO: 109

ma
GAZ: 1478
GEO: 135

mi
GAZ: 2813
GEO: 199

mn
GAZ: 1338
GEO: 130

ms
GAZ: 664
GEO: 56

mo
GAZ: 1393
GEO: 130

mt
GAZ: 271
GEO: 39

ne
GAZ: 532
GEO: 42

nv
GAZ: 687
GEO: 50

nh
GAZ: 295
GEO: 39

nj
GAZ: 2010
GEO: 177

nm
GAZ: 499
GEO: 55

ny
GAZ: 4918
GEO: 293

nc
GAZ: 2195
GEO: 212

nd
GAZ: 205
GEO: 30

oh
GAZ: 2952
GEO: 209

ok
GAZ: 1046
GEO: 76

or
GAZ: 834
GEO: 93

pa
GAZ: 3218
GEO: 206

ri
GAZ: 244
GEO: 36

sc
GAZ: 1103
GEO: 106

sd
GAZ: 222
GEO: 29

tn
GAZ: 1497
GEO: 127

tx
GAZ: 5265
GEO: 514

ut
GAZ: 588
GEO: 74

vt
GAZ: 

---

In [17]:
ESTIMATES_DATA_DICT = SF_DATA_DICT[~SF_DATA_DICT.column.isin(['FILEID', 'FILETYPE', 'CHARITER', 'SEQUENCE'])].reset_index(drop=True)
ESTIMATES_DATA_DICT.loc[ESTIMATES_DATA_DICT.column=='STUSAB', 'description'] = 'State ID'
ESTIMATES_DATA_DICT.loc[ESTIMATES_DATA_DICT.column=='LOGRECNO', 'description'] = 'Logical Record Number'
ESTIMATES_DATA_DICT.head(2)

,column,description
0,STUSAB,State ID
1,LOGRECNO,Logical Record Number


In [18]:
ESTIMATES.head(2)

,STUSAB,LOGRECNO,B01001_001,B01001_002,B01001_003,B01001_004,B01001_005,B01001_006,B01001_007,B01001_008,...,B99282_005,B99282_006,B99282_007,B99282_008,B99282_009,B99283_001,B99283_002,B99283_003,B99283_004,B99283_005
0,ak,1,731545.0,380433.0,24910.0,28292.0,25256.0,14719.0,11147.0,5181.0,...,248183,3838,248361,3849,248350,252199,7194,7194,10098,239283
1,ak,2,474992.0,240654.0,15669.0,17866.0,15164.0,9242.0,7846.0,3480.0,...,166396,2680,166398,2693,166385,169078,4779,4779,7555,159895


In [19]:
GEO_DATA_DICT = GEO_DATA_DICT[~GEO_DATA_DICT.column.isin(['FILEID', 'FILETYPE', 'CHARITER', 'SEQUENCE'])].reset_index(drop=True)
GEO_DATA_DICT.loc[GEO_DATA_DICT.column=='STUSAB', 'description'] = 'State ID'
GEO_DATA_DICT.loc[GEO_DATA_DICT.column=='LOGRECNO', 'description'] = 'Logical Record Number'
GEO_DATA_DICT.head(2)

,column,description
0,STUSAB,State ID
1,SUMLEVEL,Summary Level


In [20]:
GEO_TABLE.head(2)

,STUSAB,LOGRECNO,NECTADIV,BLANK1,CDCURR,PCI,COMPONENT,COUSUB,US,PLACE,...,AIHHTLI,STATE,COUNTY,BLKGRP,ZCTA5,STATECE,AITSCE,REGION,MEMI,BLANK4
0,ak,1,NaN,NaN,NaN,NaN,00,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ak,2,NaN,NaN,NaN,NaN,01,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
GAZ_TABLE.head(2)

,STUSAB,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,al,1001020100,9817813,28435,3.791,0.011,32.481959,-86.491338
1,al,1001020200,3325680,5669,1.284,0.002,32.475758,-86.472468


In [23]:
OUTPUTS = {
    'cen_20191_estimates_dd': ESTIMATES_DATA_DICT,
    'cen_20191_estimates': ESTIMATES,
    'cen_20191_geo_dd': GEO_DATA_DICT,
    'cen_20191_geo': GEO_TABLE,
    'cen_20191_gaz': GAZ_TABLE,
}

for f, df in OUTPUTS.items():
    print('Writing {} table'.format(f))
    #df.to_csv(os.path.join(RES_DIR, '{}.csv'.format(f)), index=None)
    df.to_pickle(os.path.join(RES_DIR, '{}.pkl'.format(f)))

Writing cen_20191_estimates_dd table
Writing cen_20191_estimates table
Writing cen_20191_geo_dd table
Writing cen_20191_geo table
Writing cen_20191_gaz table
